# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import warnings
warnings.filterwarnings("ignore")
import time
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_path = "../Resources/CityList.csv"
vacation_citydf = pd.read_csv(city_path)
vacation_citydf


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rikitea,-23.12,-134.97,74.59,82,94,9.86,PF,1603946015
1,1,qaanaaq,77.48,-69.36,7.70,72,2,8.99,GL,1603946015
2,2,punta arenas,-53.15,-70.92,46.40,65,40,28.86,CL,1603946015
3,3,port alfred,-33.59,26.89,67.26,83,98,12.06,ZA,1603946015
4,4,grindavik,63.84,-22.43,44.60,56,90,26.40,IS,1603945939
...,...,...,...,...,...,...,...,...,...,...
547,547,velykyy burluk,50.04,37.40,38.52,74,0,10.45,UA,1603946082
548,548,kyakhta,50.35,106.45,41.13,60,80,4.83,RU,1603946082
549,549,taganak,6.08,118.30,84.20,79,20,3.36,PH,1603946082
550,550,anadyr,64.75,177.48,10.40,85,40,6.71,RU,1603945914


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Find the maximum humidity from the above vacation_city dataframe and set the heat map max intesity to this value
gmaps.configure(api_key = g_key)
coordinate_list = vacation_citydf[["Lat","Lng"]]
humidity = vacation_citydf["Humidity"]
max_humid = humidity.max()
fig = gmaps.figure()
heatlayer = gmaps.heatmap_layer(coordinate_list, weights = humidity, point_radius = 5, max_intensity = max_humid)
fig.add_layer(heatlayer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Drop the rows that have null values
mycities_df = vacation_citydf.loc[(vacation_citydf["Max Temp"]>70 )& (vacation_citydf["Max Temp"] <80) &(vacation_citydf["Wind Speed"]<10) & (vacation_citydf["Cloudiness"] ==0),:]
mycities_df.dropna()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,cayenne,4.93,-52.33,78.80,94,0,9.82,GF,1603945806
22,22,busselton,-33.65,115.33,79.00,30,0,1.59,AU,1603945826
130,130,nikki,9.94,3.21,71.58,40,0,0.69,BJ,1603946029
142,142,adrar,20.50,-10.07,74.52,20,0,9.31,MR,1603945919
305,305,esmeraldas,-19.76,-44.31,71.60,78,0,1.12,BR,1603945883
307,307,sokolo,14.73,-6.12,75.83,19,0,4.72,ML,1603946052
345,345,nsanje,-16.92,35.26,77.09,63,0,5.57,MW,1603946057
355,355,mehnagar,25.88,83.12,77.00,61,0,4.70,IN,1603946058
408,408,aswan,24.09,32.91,79.05,24,0,7.92,EG,1603945952
502,502,sharjah,25.36,55.40,77.00,64,0,3.36,AE,1603946076


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#Add a column "Hotel Name" to store the nearest hotel to airport in that city
hotel_df = mycities_df
hotel_df["Hotel Name"] = ""
hotel_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
10,10,cayenne,4.93,-52.33,78.80,94,0,9.82,GF,1603945806,
22,22,busselton,-33.65,115.33,79.00,30,0,1.59,AU,1603945826,
130,130,nikki,9.94,3.21,71.58,40,0,0.69,BJ,1603946029,
142,142,adrar,20.50,-10.07,74.52,20,0,9.31,MR,1603945919,
305,305,esmeraldas,-19.76,-44.31,71.60,78,0,1.12,BR,1603945883,
307,307,sokolo,14.73,-6.12,75.83,19,0,4.72,ML,1603946052,
345,345,nsanje,-16.92,35.26,77.09,63,0,5.57,MW,1603946057,
355,355,mehnagar,25.88,83.12,77.00,61,0,4.70,IN,1603946058,
408,408,aswan,24.09,32.91,79.05,24,0,7.92,EG,1603945952,
502,502,sharjah,25.36,55.40,77.00,64,0,3.36,AE,1603946076,


In [6]:
#Set the params dict values for API call to get hotel info for the city
params = {
    "radius" : 5000,
    "type" : "lodging",
    "keyword" : "Hotel",
    "key" : g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    print(params["location"])
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params = params).json()
    try :
        hotel_df.loc[index,"Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("Hotel not found in the vicinity")
        hotel_df.loc[index,"Hotel Name"] = "NA"


4.93,-52.33
-33.65,115.33
9.94,3.21
20.5,-10.07
Hotel not found in the vicinity
-19.76,-44.31
14.73,-6.12
Hotel not found in the vicinity
-16.92,35.26
25.88,83.12
Hotel not found in the vicinity
24.09,32.91
25.36,55.4


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
10,10,cayenne,4.93,-52.33,78.80,94,0,9.82,GF,1603945806,Royal Amazonia
22,22,busselton,-33.65,115.33,79.00,30,0,1.59,AU,1603945826,Abbey Beach Resort
130,130,nikki,9.94,3.21,71.58,40,0,0.69,BJ,1603946029,Hotel John
142,142,adrar,20.50,-10.07,74.52,20,0,9.31,MR,1603945919,NA
305,305,esmeraldas,-19.76,-44.31,71.60,78,0,1.12,BR,1603945883,Parque do Avestruz Hotel Fazenda
307,307,sokolo,14.73,-6.12,75.83,19,0,4.72,ML,1603946052,NA
345,345,nsanje,-16.92,35.26,77.09,63,0,5.57,MW,1603946057,Shumba Lodge
355,355,mehnagar,25.88,83.12,77.00,61,0,4.70,IN,1603946058,NA
408,408,aswan,24.09,32.91,79.05,24,0,7.92,EG,1603945952,Sofitel Legend Old Cataract Aswan
502,502,sharjah,25.36,55.40,77.00,64,0,3.36,AE,1603946076,Crystal Plaza Hotel


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


['\n<dl>\n<dt>Name</dt><dd>Royal Amazonia</dd>\n<dt>City</dt><dd>cayenne</dd>\n<dt>Country</dt><dd>GF</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Abbey Beach Resort</dd>\n<dt>City</dt><dd>busselton</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel John</dd>\n<dt>City</dt><dd>nikki</dd>\n<dt>Country</dt><dd>BJ</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>NA</dd>\n<dt>City</dt><dd>adrar</dd>\n<dt>Country</dt><dd>MR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Parque do Avestruz Hotel Fazenda</dd>\n<dt>City</dt><dd>esmeraldas</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>NA</dd>\n<dt>City</dt><dd>sokolo</dd>\n<dt>Country</dt><dd>ML</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Shumba Lodge</dd>\n<dt>City</dt><dd>nsanje</dd>\n<dt>Country</dt><dd>MW</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>NA</dd>\n<dt>City</dt><dd>mehnagar</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Sofitel Legend Old Cataract Aswan</dd>\n<dt>City<

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info, display_info_box = True ,label = "Nearest Hotel")
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))